In [ ]:
### This script is depricated because I am able to find a better dataset from incovid19 website
### Please see another script named cases_in_district.ipynb and website: https://data.incovid19.org/ 

In [ ]:
import re
from collections import defaultdict
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
from datetime import datetime

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
hosp_dir = '/mnt/d/books/iitm/agentBased/data/tn/covid_war_room/positive_cases/'
pat_file = '/mnt/d/books/iitm/agentBased/data/tn/covid_war_room/patient_data/combined_csv.csv'

In [ ]:
skip_words = {
    'unigram': ['street', 'streets', 'road', 'roads', 'nagar', 'flat', 'school', 'apartment', 'east', 'west', 'north', 'south'],
    'default': ['s/o', 'd/o', 'w/o', 'pin', 'po', '(po)', 'part', 'opp']
}

In [ ]:
## NCDC positive cases dataset collected by Returaj

In [ ]:
# hdf = pd.read_excel(join(hosp_dir, 'NCDC_ICMR-report_1_To_50000_2021-10-21 15_29_42.xlsx'))
list_df = []
cnt = 0
types = {
    "NCDC State": "string",
    "NCDC District": "string",
    "Surveillance Id": "float64",
    "Test Id (ICMR)": "Int64",
    "Patient Age": "Int64",
    "Age In": "string",
    "Gender": "string",
    "Nationality": "string",
    "Present Village Town": "string",
    "Present Address": "string",
    "Lab Name": "string",
    "Resp Inf": "string",
    "Under Med Con": "string",
    "Travel History": "string",
    "Date of Arrival In India": "string",
    "Symptoms": "string",
    "Date of Sample Tested": "string",
    "Date of Onset of Symptoms": "string",
    "Patient Status Code": "string",
    "Facility where Patient Admitted": "string",
    "Camp Code": "float64",
    "State Code": "float64",
    "District Code": "float64",
    "Family Admitted Code": "float64",
    "Outcome Code": "string",
    "Migrated Country": "string",
    "Date Isolation": "string",
    "Total Contacts": "float64",
    "Validated": "float64"
}
for f in listdir(hosp_dir):
    filepath = join(hosp_dir, f)
    if isfile(filepath):
        tmp_df = pd.read_excel(filepath, dtype=types)
        list_df.append(tmp_df)
        cnt += 1
#     if cnt == 2:
#         break

In [ ]:
hdf = pd.concat(list_df, ignore_index=True)

In [ ]:
not_nan_df = hdf[hdf['Present Address'].notna()]
len(not_nan_df)

In [ ]:
not_nan_df.head(n=2)

In [ ]:
district_path = '/mnt/d/books/iitm/agentBased/codes/covasim/models/data/tn_districts.txt'
district_map, district_pop = {}, {}
with open(district_path, 'r') as fp:
    for line in fp:
        line = line.strip()
        if not line:
            continue
        keys = [x.strip().lower() for x in line.split(',')]
        district_pop[keys[0]] = int(keys[-3])
        d = keys[:-3]
        district_map[d[0]] = d[0]
        for i in range(1, len(d)):
            district_map[d[i]] = d[0]

In [ ]:
def create_word_vector_for_gen(sen, skip_words, from_pos=1):
    if pd.isnull(sen):
        return set()
    sen = str(sen)
    imp_keys = sen.strip().split(',')
    if len(imp_keys) == 0:
        return set()
    if len(imp_keys) > 1:
        imp_keys = imp_keys[from_pos:]
    imp_keys = [re.sub(r"[,;\-_\'\":\.\?\(\)\/\d]", " ", x).strip() for x in imp_keys]
    unigram = []
    for k in imp_keys:
        w = [x.strip().lower() for x in k.split(' ') if x.strip()]
        w = [x if x != 'st' else 'street' for x in w if (not x in skip_words['default'])]
        w = [x for x in w if len(x) > 2]
        unigram.extend(w)
    bigram = []
    for i in range(len(unigram)-1):
        bigram.append(f'{unigram[i]}_{unigram[i+1]}')
    keys = set(unigram + bigram)
    keys = set([x for x in keys if (not x in skip_words['unigram'])])
    return keys

In [ ]:
def create_word_vector_for_est(sen, skip_words):
    if pd.isnull(sen):
        return set()
    sen = str(sen)
    imp_keys = sen.strip().split(',')
    if len(imp_keys) == 0:
        return set()
    if len(imp_keys) > 1:
        imp_keys = imp_keys[-1:]
    imp_keys = [re.sub(r"[,;\-_\'\":\.\?\/\d]", " ", x).strip() for x in imp_keys]
    unigram = []
    for k in imp_keys:
        w = [x.strip().lower() for x in k.split(' ') if x.strip()]
        w = [x if x != 'st' else 'street' for x in w if (not x in skip_words['default'])]
        w = [x for x in w if len(x) > 2]
        unigram.extend(w)
    unigram = unigram[-3:]
    bigram = []
    for i in range(len(unigram)-1):
        bigram.append(f'{unigram[i]}_{unigram[i+1]}')
    keys = set(unigram + bigram)
    keys = set([x for x in keys if (not x in skip_words['unigram'])])
    return keys

In [ ]:
def get_possible_district(address, district_map, skip_words, from_pos):
    words = create_word_vector_for_gen(address, skip_words, from_pos=from_pos)
    possible_districts = [district_map[x] for x in district_map.keys() if x in words]
    return (words, possible_districts)

def population_based(possible_districts, district_pop):
    dist = ""
    for d in possible_districts:
        if (dist == "") or (district_pop[dist] < district_pop[d]):
            dist = d
    return dist

def get_district(address_districts, facility_districts, present_districts, district_pop):
    dist = []
    for ad in address_districts:
        for pd in present_districts:
            if pd == ad:
                dist.append(ad)
        for fd in facility_districts:
            if fd in dist:
                return fd
            if fd == ad:
                dist.append(fd)
    if len(dist) > 0:
        return dist[0]
    if len(address_districts) > 0:
        return population_based(address_districts, district_pop)
    elif len(present_districts) > 0:
        return population_based(present_districts, district_pop)
    else:
        return population_based(facility_districts, district_pop)


def create_word_frequency(df, district_map, district_pop, skip_words):
    word_freq, dist_cnt = {}, defaultdict(int)
    known_district, unknown_district = [], []
    for idx, row in df.iterrows():
        address_words, address_districts = get_possible_district(row['Present Address'], district_map, skip_words, from_pos=-1)

        facility_words, facility_districts = get_possible_district(row['Facility where Patient Admitted'], district_map, skip_words, from_pos=0)

        present_words, present_districts = get_possible_district(row['Present Village Town'], district_map, skip_words, from_pos=0)

        if not (address_districts or facility_districts or present_districts):
            unknown_district.append(idx)
            continue

        dist = get_district(address_districts, facility_districts, present_districts, district_pop)
        
        words = address_words.union(present_words)
        for w in words:
            if w not in word_freq:
                word_freq[w] = defaultdict(int)
            word_freq[w][dist] += 1
        dist_cnt[dist] += 1
        known_district.append((idx, dist))

#     for k, v in word_freq.items():
#         for d in v.keys():
#             v[d] /= dist_cnt[d]

#     total_cnt = sum(dist_cnt.values())
#     for k in dist_cnt.keys():
#         dist_cnt[k] /= total_cnt

    return (word_freq, dist_cnt, known_district, unknown_district)

In [ ]:
def estimate_district(df, unknown, word_prob, dist_prob, skip_words, other=False):
    estimated, new_unknown = [], []
    max_value = np.log(0.25*1e-5)
    for idx in unknown:
        if other:
            words = create_word_vector_for_est(df.loc[idx]['Address'], skip_words)
        else:
            address_words = create_word_vector_for_est(df.loc[idx]['Present Address'], skip_words)
            present_words = create_word_vector_for_est(df.loc[idx]['Present Village Town'], skip_words)
            words = address_words.union(present_words)
        dist, logit = "", -np.inf
        for d in dist_prob.keys():
            logp = 0
            for w in words:
                if (w not in word_prob) or (word_prob[w][d] == 0):
                    logp += max_value
                else:
                    logp += np.log(word_prob[w][d])
            if logp == max_value*len(words):
                continue
            logp += np.log(dist_prob[d]) # not using this because district counts are highly imbalanced
            if logp > logit:
                dist, logit = d, logp
        if dist == "":
            new_unknown.append(idx)
        else:
            estimated.append((idx, dist))
    return (estimated, new_unknown)

In [ ]:
def get_new_column_district(df, known, estimated):
    k, total_k = 0, len(known)
    e, total_e = 0, len(estimated)
    pos, total = 0, len(df)
    districts = []
    while k < total_k or e < total_e or pos < total:
        if k < total_k and known[k][0] == pos:
            districts.append(known[k][1])
            k += 1
        elif e < total_e and estimated[e][0] == pos:
            districts.append(estimated[e][1])
            e += 1
        else:
            districts.append(np.nan)
        pos += 1
    return districts

In [ ]:
def filter_words(word_prob):
    remove = []
    for k, v in word_prob.items():
        max_v = 0
        for d, p in v.items():
            max_v = max(max_v, p)
        # about 99.8% of the data is within 32 size
        if ((max_v < 1e-3 and len(k) <= 3) or (max_v < 1e-5) or (len(k) > 32)):
            remove.append(k)
    for k in remove:
        word_prob.pop(k)
        
    return word_prob

In [ ]:
word_freq, dist_cnt, known_district, unknown_district = create_word_frequency(not_nan_df, district_map, district_pop, skip_words)

In [ ]:
# 0.31598924196211614
len(known_district) / len(not_nan_df)

In [ ]:
# known_district[871:875]
# hdf.loc[[7355,7364,7370,7382]][['Present Address', 'Present Village Town', 'Facility where Patient Admitted']]

In [ ]:
## Run this block to see how district known dataset is imbalanced.

# kv = defaultdict(int)
# for idx, d in known_district:
#     kv[d] += 1
# print(kv)

# print(dist_cnt)

In [ ]:
# est, un = estimate_district(hdf, unknown_district, word_freq, dist_cnt, skip_words)

# districts = get_new_column_district(hdf, known_district, est)

# hdf['estimated_district'] = districts

In [ ]:
# len(est) + len(known_district)

In [ ]:
# hdf[hdf['estimated_district'] == 'tuticorin'][['NCDC District', 'Present Address', 'Present Village Town', 'Facility where Patient Admitted']].head(n=5)

In [ ]:
## Positive cases dataset shared by Sid
## Please be careful when you run this block of codes

In [ ]:
! wc {pat_file}

In [ ]:
types = {
    "Survellence ID": 'float64',
    "Test ID": 'Int64',
    "Lab Patient ID": "string",
    "Age": 'Int64',
    "Gender": "string",
    "Address": "string",
    "Date of Confirmation": "string",
    "Nationality": "string",
    "Total Contact": 'float64',
    "Trace Contact": 'float64',
    "Outcome Status": "string",
    "Name": "string",
    "Contact Number": 'Int64',
}
pdf = pd.read_csv(pat_file, sep=',', dtype=types)

In [ ]:
not_nan_pdf = pdf[pdf['Address'].notna()]
len(not_nan_pdf)

In [ ]:
def get_known_unknown_dist(df, district_map, district_pop, word_freq, dist_cnt, skip_words):
    known_district, unknown_district = [], []
    for idx, row in df.iterrows():
        words, address_districts = get_possible_district(row['Address'], district_map, skip_words, from_pos=-1)
        if not address_districts:
            unknown_district.append(idx)
            continue
        dist = population_based(address_districts, district_pop)
        for w in words:
            if w not in word_freq:
                word_freq[w] = defaultdict(int)
            word_freq[w][dist] += 1
        dist_cnt[dist] += 1
        known_district.append((idx, dist))

    for k, v in word_freq.items():
        for d in v.keys():
            v[d] /= dist_cnt[d]

    total_cnt = sum(dist_cnt.values())
    for k in dist_cnt.keys():
        dist_cnt[k] /= total_cnt

    return (word_freq, dist_cnt, known_district, unknown_district)

In [ ]:
word_prob, dist_prob, known, unknown = get_known_unknown_dist(not_nan_pdf, district_map, district_pop, word_freq, dist_cnt, skip_words)

In [ ]:
# 0.20802225297704172
len(known) / (len(known) + len(unknown))

In [ ]:
# kv = defaultdict(int)
# for idx, d in known:
#     kv[d] += 1
# print(kv)

# dist_prob

In [ ]:
len_cnt = defaultdict(int)
for k, v in word_prob.items():
    len_cnt[len(k)] += 1

total, cum = len(word_prob), 0
for i in range(70):
    if len_cnt[i] == 0:
        continue
    cum += len_cnt[i]
    print(f'{i}: {len_cnt[i]}, {cum / total}')

In [ ]:
from copy import deepcopy

word_prob_copy = deepcopy(word_prob)
f_word_prob = filter_words(word_prob_copy)

# ((max_v < 1e-3 and len(k) <= 3) or (max_v < 1e-5) or (len(k) > 32)):
# filtered: 400186 not_filtered: 489125

# ((max_v < 1e-3 and len(k) <= 3) or (max_v < 1e-4) or (len(k) > 32)):
# filtered: 148824 not_filtered: 489125
print(f'filtered: {len(f_word_prob)} not_filtered: {len(word_prob)}')

In [ ]:
est, un = estimate_district(pdf, unknown, f_word_prob, dist_prob, skip_words, True)

In [ ]:
# 2504669
# len(est) + len(known)

# 271409
# len(un)

In [ ]:
districts = get_new_column_district(pdf, known, est)

In [ ]:
pdf['estimated_district'] = districts

In [ ]:
pdf.head(n=5)

In [ ]:
# Filter by date

In [ ]:
dstr = '24-05-2021 09:36:59'
datetime.strptime(dstr, '%d-%m-%Y %H:%M:%S').strftime('%d-%m-%Y')

In [ ]:
def convert_to_date(date_str, date_frmt):
    try:
        d = datetime.strptime(date_str, date_frmt).strftime('%Y-%m-%d')
        return d
    except:
        return None

def get_time(df):
    dates = []
    for idx, row in df.iterrows():
        d = convert_to_date(row['Date of Confirmation'], '%d-%m-%Y %H:%M')
        if d is None:
            d = convert_to_date(row['Date of Confirmation'], '%d-%m-%Y %H:%M:%S')
        if d is None:
            d = np.nan
        dates.append(d)
    return dates

In [ ]:
dates = get_time(pdf)

In [ ]:
pdf['dates'] = dates

In [ ]:
pdf.head(n=5)

In [ ]:
# Run when needed

In [ ]:
status = pdf.groupby(['Outcome Status', 'estimated_district'])['Date of Confirmation'].agg(["size", "max"]).rename(columns={'size':'count', 'max':'to_date'}).reset_index()

In [ ]:
status[status['Outcome Status'] == 'Death']

In [ ]:
# Chengalpattu, 263, 2555
# Kancheepuram, 268, 1270
# Kanyakumari, 541, 1063
# Thiruvallur, 1059, 1866
# Tirupathur, 50, 627
# tiruvannamalai, 124, 673
# trichirappalli, 104, 1104

In [ ]:
## district and age based outcome count

In [ ]:
pdf['age_bin'] = pdf.apply(lambda r: 15 if r['Age']>=75 else int(r['Age']/5), axis=1)

In [ ]:
pdf.head(n=2)

In [ ]:
# save pdf dataframe

In [ ]:
pdf_file = '/mnt/d/books/iitm/agentBased/data/tn/covid_war_room/patient_data/refactored.csv'
pdf.to_csv(pdf_file, sep=',', index=False, header=True)

In [ ]:
# All main computation happens from here. Please run above if you want to verify else proceed from here !!

In [ ]:
# read pdf dataframe
pdf_file = '/mnt/d/books/iitm/agentBased/data/tn/covid_war_room/patient_data/refactored.csv'
types = {
    "Survellence ID": 'float64',
    "Test ID": 'Int64',
    "Lab Patient ID": "string",
    "Age": 'Int64',
    "Gender": "string",
    "Address": "string",
    "Date of Confirmation": "string",
    "Nationality": "string",
    "Total Contact": 'float64',
    "Trace Contact": 'float64',
    "Outcome Status": "string",
    "Name": "string",
    "Contact Number": 'Int64',
    "estimated_district": "string",
    "age_bin": "Int64",
    "dates": "string"
}
pdf = pd.read_csv(pdf_file, sep=',', dtype=types)

In [ ]:
pdf.head(n=2)

In [ ]:
# not_na = pdf[pdf['estimated_district'].notna()]
from_aug = pdf[(pdf['dates'] >= '2021-08-01') & pdf['estimated_district'].notna()]

In [ ]:
age_count_by_date = from_aug.groupby(['estimated_district', 'Outcome Status', 'age_bin', 'dates'])['Test ID'].agg(['size']).rename(columns={'size':'count'}).reset_index()

In [ ]:
age_count_by_date.head(n=2)

In [ ]:
## Using cnt_district we know that start_time = '2021-08-01' to end_time = '2021-10-08'
cnt_district = {}
districts = age_count_by_date.estimated_district.unique()
for d in districts:
    tmp = age_count_by_date[(age_count_by_date['estimated_district']==d)].dates.unique()
    cnt_district[d] = (min(tmp), max(tmp), len(tmp))
cnt_district

In [ ]:
exp_cnt = from_aug.groupby(['estimated_district', 'Outcome Status', 'age_bin'])['Test ID'].agg(['size']).rename(columns={'size':'count'}).reset_index()

In [ ]:
exp_cnt['expected_infection'] = exp_cnt['count'].apply(lambda c: c/69)

In [ ]:
exp_cnt.head(n=5)

In [ ]:
exp_cnt['Outcome Status'].unique()

In [ ]:
exp_cnt[(exp_cnt['estimated_district'] == 'chennai') & (exp_cnt['Outcome Status'] == 'Discharge')]